<a href="https://colab.research.google.com/github/MadhuryaPasan/Qwen3-ToolCalling-FineTune/blob/main/Notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth

In [3]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen3-VL-2B-Instruct-FP8",
    load_in_4bit = False,
    load_in_8bit = True,
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

==((====))==  Unsloth 2026.2.1: Fast Qwen3_Vl patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/782 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [4]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 8,
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

In [5]:
from datasets import load_dataset
dataset = load_dataset("Salesforce/xlam-function-calling-60k", split = "train")

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

xlam_function_calling_60k.json:   0%|          | 0.00/96.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [3]:
dataset

Dataset({
    features: ['id', 'query', 'answers', 'tools'],
    num_rows: 60000
})

In [ ]:
dataset[1]

{'id': 1,
 'query': "I need to understand the details of the Ethereum blockchain for my cryptocurrency project. Can you fetch the details for 'ethereum'?",
 'answers': '[{"name": "web_chain_details", "arguments": {"chain_slug": "ethereum"}}]',
 'tools': '[{"name": "peers", "description": "Retrieves a list of company peers given a stock symbol.", "parameters": {"symbol": {"description": "The stock symbol for the company.", "type": "str", "default": ""}}}, {"name": "web_chain_details", "description": "python", "parameters": {"chain_slug": {"description": "The slug identifier for the blockchain (e.g., \'ethereum\' for Ethereum mainnet).", "type": "str", "default": "ethereum"}}}]'}

In [5]:
# @title
# def convert_xlam_to_conversation(sample):
#     # 1. Define the Standard xLAM System Instruction
#     system_instruction = (
#         "You are an expert in tool calling. You will be provided with a list of tools and a user query. Your goal is to output the correct tool call in JSON format."
#     )

#     # 2. Format the User prompt with clear delimiters
#     # Standard xLAM uses [BEGIN OF TOOL_DEFINITION] or similar structures
#     user_prompt = (
#         f"[BEGIN OF TOOL_DEFINITION]\n{sample['tools']}\n[END OF TOOL_DEFINITION]\n"
#         f"Query: {sample['query']}"
#     )

#     # 3. Format the Assistant output
#     # The 'answers' column in xLAM is already a JSON string
#     assistant_output = sample['answers']

#     return {
#         "messages": [
#             {"role": "system", "content": system_instruction},
#             {"role": "user", "content": user_prompt},
#             {"role": "assistant", "content": assistant_output}
#         ]
#     }

# converted_dataset = dataset.map(convert_xlam_to_conversation,remove_columns=dataset.column_names)

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [6]:
# @title
# from unsloth.chat_templates import get_chat_template

# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "qwen-3", # Qwen3 uses the same underlying ChatML structure
# )

# def formatting_prompts_func(sample):
#     # Standard xLAM setup for an Instruct model
#     messages = [
#         {"role": "system", "content": "You are a helpful assistant with access to tools. Output JSON tool calls."},
#         {"role": "user", "content": f"Tools:\n{sample['tools']}\n\nQuery: {sample['query']}"},
#         {"role": "assistant", "content": sample['answers']}
#     ]
#     return {"text": tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)}

# # Apply to your xLAM dataset
# converted_dataset = dataset.map(formatting_prompts_func)

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [8]:
converted_dataset[1]

{'id': 1,
 'query': "I need to understand the details of the Ethereum blockchain for my cryptocurrency project. Can you fetch the details for 'ethereum'?",
 'answers': '[{"name": "web_chain_details", "arguments": {"chain_slug": "ethereum"}}]',
 'tools': '[{"name": "peers", "description": "Retrieves a list of company peers given a stock symbol.", "parameters": {"symbol": {"description": "The stock symbol for the company.", "type": "str", "default": ""}}}, {"name": "web_chain_details", "description": "python", "parameters": {"chain_slug": {"description": "The slug identifier for the blockchain (e.g., \'ethereum\' for Ethereum mainnet).", "type": "str", "default": "ethereum"}}}]',
 'text': '<|im_start|>system\nYou are a helpful assistant with access to tools. Output JSON tool calls.<|im_end|>\n<|im_start|>user\nTools:\n[{"name": "peers", "description": "Retrieves a list of company peers given a stock symbol.", "parameters": {"symbol": {"description": "The stock symbol for the company.", "

In [ ]:
from unsloth.chat_templates import get_chat_template

# 1. Initialize the tokenizer with the correct Qwen3 template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-3", # Use the built-in unsloth qwen-3 template
)

def formatting_prompts_func(sample):
    import json

    # Qwen3 template expects tools in a specific list of dictionaries
    # Convert the xLAM tool string into a Python list
    tools = json.loads(sample["tools"])

    # Convert xLAM answers into the 'tool_calls' format the template expects
    # xLAM answers: '[{"name": "...", "arguments": {...}}]'
    ans_list = json.loads(sample["answers"])
    tool_calls = []
    for a in ans_list:
        tool_calls.append({
            "type": "function",
            "function": {
                "name": a["name"],
                "arguments": a["arguments"]
            }
        })

    # Construct the message history
    messages = [
        {"role": "user", "content": sample["query"]},
        {"role": "assistant", "tool_calls": tool_calls}
    ]

    # APPLY THE TEMPLATE
    # tools=tools will trigger the <tools> block in the Jinja template
    text = tokenizer.apply_chat_template(
        messages,
        tools = tools,
        tokenize = False,
        add_generation_prompt = False,
    )

    return { "text" : text }

dataset = dataset.map(formatting_prompts_func)